In [ ]:
#%matplotlib widget
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date,timedelta
import json
import numpy as np
from ipynb.fs.full.my_functions import *
import time
import math
from math import cos, sin
import statistics

In [ ]:
ET_DATA_PATH = r'E:\MasterMax\sample_code\EEDA_Data\EEDA_Data\EyeTracking'
SCENE_EVENT_DATA_PATH = r'E:\MasterMax\sample_code\EEDA_Data\EEDA_Data\SceneData'
STEERING_DATA_PATH = r'E:\MasterMax\sample_code\EEDA_Data\EEDA_Data\Input'
PARTICIPANT_DATA_PATH = r'E:\MasterMax\sample_code\EEDA_Data\EEDA_Data\ParticipantCalibrationData'
PROCESSED_DATA_PATH = '..\csv_et'
COMBINED_DATA_PATH = '..\et_comb'
EVENT_DURATION_DATA_PATH = '..\Event_dur'
ONLY_EVENTS_DATA_PATH = '..\MA_Plots_Manual' # Only_Events - Autonom
TAXI_PATH = '..\Taxi_drive'
EVENTS_OBJECT_PATH = '..\Evt_Objects'
Man_Path = '..\Manual_Scenes'
Auto_Path = '..\Autonomous_scenes'

PLOT_PATH = '..\PLOTS\data_quality'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(COMBINED_DATA_PATH),exist_ok=True)
os.makedirs(os.path.dirname(PLOT_PATH), exist_ok=True)

## Data Filter

### Data Preprocessing 
Reads all data and collects them in the respective scenes.

In [ ]:
# Loads Participants Files
Participant_files = glob.glob(f'{PARTICIPANT_DATA_PATH}/*.txt')
files = glob.glob(f'{ONLY_EVENTS_DATA_PATH}\*.csv')


df = pd.DataFrame()
# Filters for EyeValidation error above 1.5
df = checkEyeError(Participant_files[:])
# checks if Participants made it thorugh till the end
df = checkIncomplete(ET_DATA_PATH,df)
display(df)
# Filter for the wanted condition
Conditionname = 'Autonomous' # Autonomous, ManualDriving, SemiAutonomous, TaxiDriver
df = selectConditions(df,Conditionname)
display(df)

### Main Preprocessing
Saving the data of all participants surviving the filtering to scene specific files

In [ ]:
df_A = pd.DataFrame()
# timekeeper for overall process
start2 = time.process_time()
# Goes through each participants 
for index, row in df.iterrows():
    
    name = row[0]
    print(name)
    #timekeeper per iteration
    start = time.process_time()
    # Select desired Scenes
    scenes = ['CountryRoad',
              'MountainRoad',
              'Westbrueck']

    neList = list()
    # Get File lists
    CompL = SelectAllFilesOfSameName(name,ET_DATA_PATH)
    CompLSteer = SelectAllFilesOfSameName(name,STEERING_DATA_PATH)
    CompLEvents = SelectAllFilesOfSameName(name,SCENE_EVENT_DATA_PATH)
    CompLEventCuts = SelectAllCSVOfSameName(TAXI_PATH)
    # Goes through each scene for each particpant
    for i in scenes:
        # get all needed Eyetracking files
        for b in CompL:
            if i in b:
                neList = list()
                neList.append(b)
                et_df = LoadEyetrackingData(neList)
                #print("Loaded Eyetracking")
        # get all needed steering files
        for c in CompLSteer:
            if i in c:
                neList = list()
                neList.append(c)
                st_df = ReadSteeringData(neList)
        # Possible error in some files when combining steering and eyetracking data
        try:
            et_df = CombineSteeringAndET(et_df,st_df)
        except:
            print("error")
            continue
        # get all needed Event files
        for d in CompLEvents:
            if i in d:
                neList = list()
                neList.append(d)
                evt_df,mvmt_df = ReadEventData(neList)
                
        # Calculate the Theta values
        et_df = EyeThetaCalculation(et_df)
        
        # old function for the Event snippets
        # evt_df = CalculateEventTime(et_df,evt_df)
        # Claculates 'Distance' values = Euklidian distance summ of x and z position of the car
        et_df = Distance(et_df)
        
        # check if a scene-file is already existing, if not create it
        path = os.path.join(TAXI_PATH,Conditionname+'_'+ i+'.csv')
        if path in CompLEventCuts:

            O_df = pd.read_csv(path)
            print("Found file ", i)
        else :
            O_df = pd.DataFrame()
            print("New File")#
        # add new participant to others
        O_df = pd.concat([O_df, et_df], ignore_index=True)
        # Save the file to ONLY_EVENTS_DATA_PATH
        O_df.to_csv(path, index = False)
    
    print("One Iteration needs: ",time.process_time() - start)
    
    print(index)
print("Alles in: ",time.process_time() - start2)


### Comparison of Gaze and Visualization

In [ ]:
filesT = glob.glob(f'{TAXI_PATH}\*.csv')#
files = glob.glob(f'{ONLY_EVENTS_DATA_PATH}\*.csv')#Man_Path
filesM = glob.glob(f'{Man_Path}\*.csv')#Man_Path
print(files)
print(filesM)
for fi in files[1:2]:
    df_A = pd.read_csv(fi)
    df_A = RotationStuff(df_A) # Calculate average by distance, bin size is 2
    #uids = list(df_A.uid.unique()) 
    #for u in uids[:1]:
        #df2 = df_A.loc[df_A['uid'] == u]
    
    #df2 = GetAverage(df2,"Distance",2)
    df_A = GetAverage(df_A,"Distance",2)
for fi in filesM[1:2]:
    df_M = pd.read_csv(fi)
    df_M = RotationStuff(df_M) 
    #uids = list(df_M.uid.unique()) 
    #for u in uids[:1]:
        #df1 = df_M.loc[df_M['uid'] == u]
    # Calculate average by distance, bin size is 2
    df_M = GetAverage(df_M,"Distance",2)
    #df1 = GetAverage(df1,"Distance",2)
for fi in filesT[1:2]:
    df_T = pd.read_csv(fi)
    df_T = RotationStuff(df_T) 
    #uids = list(df_T.uid.unique()) 
    #for u in uids[:1]:
        #df1 = df_T.loc[df_M['uid'] == u]
    # Calculate average by distance, bin size is 2
    df_T = GetAverage(df_T,"Distance",2)
    #df1 = GetAverage(df1,"Distance",2)
df_A['eye_theta_h_HEAD'] = df_A['eye_theta_h_HEAD']*180/math.pi
df_M['eye_theta_h_HEAD'] = df_M['eye_theta_h_HEAD']*180/math.pi
df_T['eye_theta_h_HEAD'] = df_T['eye_theta_h_HEAD']*180/math.pi
#df1['eye_theta_h_HEAD'] = df1['eye_theta_h_HEAD']*180/math.pi
#df2['eye_theta_h_HEAD'] = df2['eye_theta_h_HEAD']*180/math.pi
print("A")
sns.set(context = "talk", style="ticks", palette="muted", font_scale=2,)
fig, ax = plt.subplots(figsize=(80,30),ncols=3)#
cm = plt.cm.get_cmap('RdYlGn')
x2 = df_A['eye_theta_h_HEAD']
x1 = df_M['eye_theta_h_HEAD']
x3 = df_T['eye_theta_h_HEAD']
#x4 = df2['eye_theta_h_HEAD']
df_A.plot(kind="scatter",title = 'Averaged HMD and Eye Orientation Combined in Autonomous Condition of Mountainroad', x='CarPosition_x',y= 'CarPosition_z',ax=ax[0],lw = 15,zorder=200,c=x2, vmin=-20, vmax=20, s=10, cmap=cm)
df_M.plot(kind="scatter",title = 'Averaged HMD and Eye Orientation Combined in Manual Condition of Mountainroad', x='CarPosition_x',y= 'CarPosition_z',ax=ax[1],lw = 15,zorder=200,c=x1, vmin=-20, vmax=20, s=10, cmap=cm)
df_T.plot(kind="scatter",title = 'Averaged HMD and Eye Orientation Combined in Taxi Driver Condition of Mountainroad', x='CarPosition_x',y= 'CarPosition_z',ax=ax[2],lw = 15,zorder=200,c=x3, vmin=-20, vmax=20, s=10, cmap=cm)
#df1.plot(kind="scatter",title = 'Single Participant HMD and Eye Orientation Combined in Manual Condition of Mountainroad', x='CarPosition_x',y= 'CarPosition_z',ax=ax[1][1],lw = 15,zorder=200,c=x3, vmin=-20, vmax=20, s=10, cmap=cm)
#df2.plot(kind="scatter",title = 'Single Participant HMD and Eye Orientation Combined in Autonomous Condition of Mountainroad', x='CarPosition_x',y= 'CarPosition_z',ax=ax[0][1],lw = 15,zorder=200,c=x4, vmin=-20, vmax=20, s=10, cmap=cm)

ax[0].set_xlabel("X-Coordinates")
ax[0].set_ylabel("Z-Coordinates")
ax[1].set_xlabel("X-Coordinates")
ax[1].set_ylabel("Z-Coordinates")
ax[2].set_xlabel("X-Coordinates")
ax[2].set_ylabel("Z-Coordinates")